---
### Sampler와 collate_fn

In [2]:
import torch
import numpy as np
from torch.utils.data import Dataset, ConcatDataset, Sampler, RandomSampler, BatchSampler
import transformers
import tokenizers

In [3]:
class MapDataset(Dataset):
    def __len__(self):
        return 10
    
    def __getitem__(self, idx):
        return {"input":torch.tensor([idx, 2*idx, 3*idx], 
                                     dtype=torch.float32),
                "label": torch.tensor(idx, 
                                     dtype=torch.float32)}


class CSBDataset(Dataset):
    def __len__(self):
        return 10
    
    def __getitem__(self, idx):
        return torch.tensor([idx, 2*idx, 3*idx], 
                                     dtype=torch.float32)


map_dataset = MapDataset()
csb_dataset = CSBDataset()

In [4]:
point_sampler = RandomSampler(map_dataset)
batch_sampler = BatchSampler(point_sampler, 3, False)
dataloader = torch.utils.data.DataLoader(map_dataset,
                                         batch_sampler=batch_sampler)
for data in dataloader:
    print(data['input'].shape, data['label'])


point_sampler = RandomSampler(map_dataset)
batch_sampler = BatchSampler(point_sampler, 4, True)
### 이렇게 하면 에러 뜬다. 
### batch_sampler는 batch_sampler 인자에 넣어주어야한다. sampler 인자에 넣어주면 에러뜬다 무조건.  

dataloader = torch.utils.data.DataLoader(map_dataset,sampler=batch_sampler)


point_sampler = RandomSampler(map_dataset)
batch_sampler = BatchSampler(point_sampler, 4, False)
### 이렇게 해도 에러 뜬다
### batch_sampler와 batch_size는 같이 쓸 수 없다. 
dataloader = torch.utils.data.DataLoader(map_dataset,batch_sampler=batch_sampler, batch_size=3)


torch.Size([3, 3]) tensor([8., 1., 6.])
torch.Size([3, 3]) tensor([0., 7., 9.])
torch.Size([3, 3]) tensor([4., 2., 3.])
torch.Size([1, 3]) tensor([5.])


ValueError: batch_sampler option is mutually exclusive with batch_size, shuffle, sampler, and drop_last

In [5]:
dataloader = torch.utils.data.DataLoader(map_dataset,
                                         batch_size=3)

csb_dataloader = torch.utils.data.DataLoader(csb_dataset, batch_size = 3) 

for data, data2 in zip(dataloader, csb_dataloader):
    print(data, data2)

### 찾아보니 dataset이 dict을 return할 경우 내 예상은 batch로 묶일때 dictionary 여러개가 나올줄 알았지만, 그게 아니고 
### default collate fn이 있어서 dict안의 값이 여러개로 묶인다. default_collate_fn이 dictionary안에 넣어준다고 한다. 

{'input': tensor([[0., 0., 0.],
        [1., 2., 3.],
        [2., 4., 6.]]), 'label': tensor([0., 1., 2.])} tensor([[0., 0., 0.],
        [1., 2., 3.],
        [2., 4., 6.]])
{'input': tensor([[ 3.,  6.,  9.],
        [ 4.,  8., 12.],
        [ 5., 10., 15.]]), 'label': tensor([3., 4., 5.])} tensor([[ 3.,  6.,  9.],
        [ 4.,  8., 12.],
        [ 5., 10., 15.]])
{'input': tensor([[ 6., 12., 18.],
        [ 7., 14., 21.],
        [ 8., 16., 24.]]), 'label': tensor([6., 7., 8.])} tensor([[ 6., 12., 18.],
        [ 7., 14., 21.],
        [ 8., 16., 24.]])
{'input': tensor([[ 9., 18., 27.]]), 'label': tensor([9.])} tensor([[ 9., 18., 27.]])


### 이거 확인한번!!!

In [6]:
map_dataset[0]

{'input': tensor([0., 0., 0.]), 'label': tensor(0.)}

In [7]:
point_sampler = RandomSampler(map_dataset)
batch_sampler = BatchSampler(point_sampler, 3, False)


### 보니까 collate_fn의 경우에는 리스트 이용해서 for 문 하는게 제일 좋은듯하다. 
def custom_fn(batch):
    x = [data['input'] +100 for data in batch]
    y = [data['label'] +50 for data in batch]

    return x, y


dataloader = torch.utils.data.DataLoader(map_dataset,
                                         batch_sampler=batch_sampler,
                                         collate_fn = custom_fn)


In [8]:
for data in dataloader:
    print(data)

([tensor([104., 108., 112.]), tensor([107., 114., 121.]), tensor([101., 102., 103.])], [tensor(54.), tensor(57.), tensor(51.)])
([tensor([100., 100., 100.]), tensor([102., 104., 106.]), tensor([109., 118., 127.])], [tensor(50.), tensor(52.), tensor(59.)])
([tensor([108., 116., 124.]), tensor([105., 110., 115.]), tensor([103., 106., 109.])], [tensor(58.), tensor(55.), tensor(53.)])
([tensor([106., 112., 118.])], [tensor(56.)])
